In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from IPython.display import display
from nltk import TweetTokenizer
import pickle
from gensim.models.keyedvectors import KeyedVectors
import time
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras import utils
from keras.layers import Embedding, Flatten, Concatenate, concatenate, Input
from keras.layers import Conv1D, GlobalMaxPooling1D, Conv2D
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import callbacks
from keras.preprocessing import sequence
from keras import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from gensim.models import Word2Vec

Using TensorFlow backend.


In [2]:
%%time

#labels
dic_aggression_level = {
    'NAG' : 1,
    'CAG' : 2,
    'OAG' : 3
}

dic_reverse_aggression_level = {}
for i in dic_aggression_level:
    dic_reverse_aggression_level[dic_aggression_level[i]] = i
    
print(dic_aggression_level, '\n', dic_reverse_aggression_level)

#train data
train_pd = shuffle(pd.concat((pd.read_csv("train.csv")[['Data', 'Label']], pd.read_csv("valid.csv")[['Data', 'Label']])), random_state=20)
train_pd['Label_num'] = train_pd.Label.map(dic_aggression_level)

#test data
test_fb_pd = shuffle(pd.read_csv("test_fb.csv")[['Data', 'Label']], random_state=20)
test_fb_pd['Label_num'] = test_fb_pd.Label.map(dic_aggression_level)
test_tw_pd = shuffle(pd.read_csv("test_tw.csv")[['Data', 'Label']], random_state=20)
test_tw_pd['Label_num'] = test_tw_pd.Label.map(dic_aggression_level)

#word2vec
word2vec_model = KeyedVectors.load("glove.twitter.27B/glove.twitter.27B.200d.txt.vector.bin")
vocabulary = set(word2vec_model.wv.vocab.keys())
# tokenizer = TweetTokenizer()
# def create_tokens(x):
#     return tokenizer.tokenize(x.lower())
# word2vec_model = Word2Vec(list(map(create_tokens, train_pd['Data'])), size=200, window=5, min_count=1, workers=50)
# vocabulary = set(word2vec_model.wv.vocab.keys())

print("TRAIN DATA")
print(train_pd.Label.value_counts())
display(train_pd.head(10))

print("\n\n\nTEST DATA")
print(test_fb_pd.Label.value_counts())
display(test_fb_pd.head(10))

{'NAG': 1, 'OAG': 3, 'CAG': 2} 
 {1: 'NAG', 2: 'CAG', 3: 'OAG'}
TRAIN DATA
NAG    6285
CAG    5297
OAG    3419
Name: Label, dtype: int64


/opt/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


,Data,Label,Label_num
2930,Focus on making cash available then only peo...,CAG,2
5103,She's so ignorant Megha Mukherji,OAG,3
5090,"Sonia I am holding Rel cap 430, please suggest...",NAG,1
9074,why dont u make ur room sound proof..simple,OAG,3
6769,Showing everything and saying bold...,CAG,2
140,Then what happens in pantry coach dedicated fo...,OAG,3
1756,We should respect every religion. May be he wa...,OAG,3
7272,Car is good.. bt i must say.. i only heard 'aa...,NAG,1
10305,friends we have to understand the ground reali...,CAG,2
45,Bad...........,CAG,2





TEST DATA
NAG    630
OAG    144
CAG    142
Name: Label, dtype: int64


,Data,Label,Label_num
156,What will be done to the money we have ?,NAG,1
211,Unchange the rapo rate could lead the stagnate...,NAG,1
13,PK Movie Bhagawan Shiv Ko Aapman Kiya.. I Ha...,NAG,1
798,Thousands of people have died due to bandhs an...,CAG,2
640,Worst F.M ever.what about 5 lakhs tax limit? ...,NAG,1
568,Pak army rape modi daughter and go back. Now m...,NAG,1
321,I am clean without cash as transactions are th...,NAG,1
119,I have 5000 shares of Pnb @75.90 please tell m...,NAG,1
820,these bhagwa terrorists can't digest their mea...,OAG,3
721,Worst Decision and very worst implementation. ...,OAG,3


CPU times: user 3.93 s, sys: 1.13 s, total: 5.06 s
Wall time: 8.3 s


In [3]:
%%time
tweeter = TweetTokenizer()

def get_sentence_vector(sentence, num_words=-1):
    global random_listy
    listy = []
    for token in tweeter.tokenize(sentence):
        if token in vocabulary:
            listy.append(word2vec_model[token])  
    
#     random_listy.append(len(listy))
    
    if num_words != -1:
        listy = listy[:num_words]
    if len(listy) < num_words:
        listy = listy + [[0]*word2vec_model['king'].shape[0]]*(num_words-len(listy))
    
    return np.array(listy).reshape((1, -1))[0]
         
# number_of_words_per_sentence = 85
number_of_words_per_sentence = sum([len(tweeter.tokenize(sentence)) for sentence in train_pd['Data']]) // train_pd['Data'].shape[0]
X_train_ = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in train_pd['Data']])
y_train_ = utils.to_categorical(train_pd['Label_num']-1)
X_test_fb = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in test_fb_pd['Data']])
X_test_tw = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in test_tw_pd['Data']])
y_test_fb = test_fb_pd['Label_num']
y_test_tw = test_tw_pd['Label_num']
num_classes = y_train_.shape[1]
print("There are {} words per sentence.".format(number_of_words_per_sentence))
print("X_train: {}\ny_train: {}".format(X_train_.shape, y_train_.shape))
print("X_test_fb: {}\ny_test_fb: {}".format(X_test_fb.shape, y_test_fb.shape))
print("X_test_tw: {}\ny_test_tw: {}".format(X_test_tw.shape, y_test_tw.shape))

There are 27 words per sentence.
X_train: (15001, 5400)
y_train: (15001, 3)
X_test_fb: (916, 5400)
y_test_fb: (916,)
X_test_tw: (1257, 5400)
y_test_tw: (1257,)
CPU times: user 5.88 s, sys: 392 ms, total: 6.27 s
Wall time: 6.37 s


In [ ]:
save_folder = "results/NN_word2vec/"

#callbacks
callback_list = []
callback_list.append(callbacks.EarlyStopping(monitor='val_acc', min_delta=0.003, patience=3, verbose=0, mode='auto', baseline=None))


def createCNNModel(layer_node_list):
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
#     model = Sequential()
#     models = []
#     for kernel_size in range(200, 1001, 200):
# #         model_temp = Sequential()
#         models.append(Conv1D(filters=8, #!!!!!!!!!!!!!!!!!!!
#                          kernel_size=kernel_size,
#                          padding='valid',
#                          activation='relu',
#                          strides=200,
#                          input_shape=(X_train.shape[1], 1)
#                          ))
#     #     model.add(GlobalMaxPooling1D())
    
# #         models.append(model_temp)
#     print(models)
# #     model = Sequential()
#     model.add(Concatenate(models))
#     model.add(Flatten())
#     model.add(Dense(200))
#     model.add(Dropout(0.5))
#     model.add(Activation('relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(50))
#     model.add(Activation('sigmoid'))
#     model.add(Dense(3, activation='softmax'))
# #     model.summary()
#     model.compile(loss='mean_squared_error',
#                   optimizer='adam',
#                   metrics=['acc',metrics.binary_accuracy])
#     return model
    input_crap = Input(shape=(X_train.shape[1], 1))
    convo_layers = [Conv1D(filters=8, #!!!!!!!!!!!!!!!!!!!
                         kernel_size=kernel_size,
                         padding='same',
                         activation='relu',
                         strides=200,
                         input_shape=(X_train.shape[1], 1),
                          )(input_crap) for kernel_size in range(200, 1001, 200)]
    merged_layer = concatenate(convo_layers)
    flattened_layer = Flatten()(merged_layer)
    fc1 = Dense(layer_node_list[0], activation='relu')(flattened_layer)
    dropout1 = Dropout(0.5)(fc1)
    fc2 = Dense(layer_node_list[1], activation='sigmoid') (dropout1)
    dropout2 = Dropout(0.3)(fc2)
    fc3 = Dense(3, activation='softmax')(dropout2)
    model = Model(inputs=input_crap, outputs=fc3)
    model.summary()
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model
    
    

# create our CNN model
X_train, X_valid, y_train, y_valid = train_test_split(X_train_.reshape(X_train_.shape[0], -1, 1), y_train_, test_size=0.2)
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
for nodes1 in range(100, 401, 20):
    for nodes2 in range(20, 401, 20):
        print(nodes1, nodes2)
        model = createCNNModel([nodes1, nodes2])
        model.fit(X_train, y_train, validation_data=(X_valid, y_valid), nb_epoch=15, batch_size=128,
                 callbacks=callback_list)
        # Final evaluation of the model
        print("\n\n")
        y_pred_fb = model.predict(X_test_fb.reshape(X_test_fb.shape[0], -1, 1))
        y_pred_tw = model.predict(X_test_tw.reshape(X_test_tw.shape[0], -1, 1))
        y_pred_fb = np.array([y_pred_fb[i].argmax()+1 for i in range(len(y_pred_fb))])
        y_pred_tw = np.array([y_pred_tw[i].argmax()+1 for i in range(len(y_pred_tw))])

        print("FB: {}, {} {}".format(accuracy_score(y_test_fb, y_pred_fb), f1_score(y_test_fb, y_pred_fb, average='weighted'),
                                 confusion_matrix(y_test_fb, y_pred_fb)))
        print("TW: {}, {} {}".format(accuracy_score(y_test_tw, y_pred_tw), f1_score(y_test_tw, y_pred_tw, average='weighted'),
                                 confusion_matrix(y_test_tw, y_pred_tw)))
        del model

(12000, 5400, 1) (12000, 3) (3001, 5400, 1) (3001, 3)
100 20
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5400, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 27, 8)        1608        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 27, 8)        3208        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 27, 8)        4808        input_1[0][0]                    
________________________________________________

/opt/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:74: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 4s 330us/step - loss: 0.2148 - acc: 0.4267 - binary_accuracy: 0.6697 - val_loss: 0.1954 - val_acc: 0.4955 - val_binary_accuracy: 0.7007
Epoch 2/15
12000/12000 [==============================] - 3s 276us/step - loss: 0.1961 - acc: 0.5036 - binary_accuracy: 0.7020 - val_loss: 0.1903 - val_acc: 0.5275 - val_binary_accuracy: 0.7055
Epoch 3/15
12000/12000 [==============================] - 3s 275us/step - loss: 0.1887 - acc: 0.5316 - binary_accuracy: 0.7127 - val_loss: 0.1877 - val_acc: 0.5385 - val_binary_accuracy: 0.7063
Epoch 4/15
12000/12000 [==============================] - 3s 270us/step - loss: 0.1821 - acc: 0.5613 - binary_accuracy: 0.7226 - val_loss: 0.1864 - val_acc: 0.5468 - val_binary_accuracy: 0.7152
Epoch 5/15
12000/12000 [==============================] - 3s 277us/step - loss: 0.1734 - acc: 0.5948 - binary_accuracy: 0.7391 - val_loss: 0.1861 - val_acc: 0.5438 - val_binar

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 4s 336us/step - loss: 0.2170 - acc: 0.4313 - binary_accuracy: 0.6606 - val_loss: 0.2010 - val_acc: 0.4865 - val_binary_accuracy: 0.6861
Epoch 2/15
12000/12000 [==============================] - 3s 266us/step - loss: 0.1971 - acc: 0.5009 - binary_accuracy: 0.6998 - val_loss: 0.1897 - val_acc: 0.5145 - val_binary_accuracy: 0.7063
Epoch 3/15
12000/12000 [==============================] - 3s 268us/step - loss: 0.1874 - acc: 0.5353 - binary_accuracy: 0.7139 - val_loss: 0.1868 - val_acc: 0.5408 - val_binary_accuracy: 0.7100
Epoch 4/15
12000/12000 [==============================] - 3s 268us/step - loss: 0.1793 - acc: 0.5680 - binary_accuracy: 0.7289 - val_loss: 0.1867 - val_acc: 0.5318 - val_binary_accuracy: 0.7142
Epoch 5/15
12000/12000 [==============================] - 3s 268us/step - loss: 0.1692 - acc: 0.6082 - binary_accuracy: 0.7476 - val_loss: 0.1882 - val_acc: 0.5328 - val_binar

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 4s 342us/step - loss: 0.2162 - acc: 0.4352 - binary_accuracy: 0.6648 - val_loss: 0.1944 - val_acc: 0.4992 - val_binary_accuracy: 0.6997
Epoch 2/15
12000/12000 [==============================] - 3s 273us/step - loss: 0.1978 - acc: 0.4976 - binary_accuracy: 0.6959 - val_loss: 0.1895 - val_acc: 0.5142 - val_binary_accuracy: 0.7053
Epoch 3/15
12000/12000 [==============================] - 3s 275us/step - loss: 0.1880 - acc: 0.5312 - binary_accuracy: 0.7124 - val_loss: 0.1865 - val_acc: 0.5452 - val_binary_accuracy: 0.7131
Epoch 4/15
12000/12000 [==============================] - 3s 273us/step - loss: 0.1789 - acc: 0.5678 - binary_accuracy: 0.7296 - val_loss: 0.1893 - val_acc: 0.5368 - val_binary_accuracy: 0.7089
Epoch 5/15
12000/12000 [==============================] - 3s 273us/step - loss: 0.1692 - acc: 0.6086 - binary_accuracy: 0.7477 - val_loss: 0.1889 - val_acc: 0.5405 - val_binar

...

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 10s 806us/step - loss: 0.2209 - acc: 0.4130 - binary_accuracy: 0.6580 - val_loss: 0.2013 - val_acc: 0.4892 - val_binary_accuracy: 0.6961
Epoch 2/15
12000/12000 [==============================] - 4s 310us/step - loss: 0.1986 - acc: 0.4952 - binary_accuracy: 0.7004 - val_loss: 0.1939 - val_acc: 0.4972 - val_binary_accuracy: 0.7025
Epoch 3/15
12000/12000 [==============================] - 4s 308us/step - loss: 0.1912 - acc: 0.5265 - binary_accuracy: 0.7108 - val_loss: 0.1894 - val_acc: 0.5272 - val_binary_accuracy: 0.7075
Epoch 4/15
12000/12000 [==============================] - 4s 327us/step - loss: 0.1852 - acc: 0.5430 - binary_accuracy: 0.7179 - val_loss: 0.1875 - val_acc: 0.5382 - val_binary_accuracy: 0.7079
Epoch 5/15
12000/12000 [==============================] - 4s 321us/step - loss: 0.1799 - acc: 0.5692 - binary_accuracy: 0.7269 - val_loss: 0.1877 - val_acc: 0.5332 - val_bina

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 11s 884us/step - loss: 0.2155 - acc: 0.4381 - binary_accuracy: 0.6655 - val_loss: 0.1975 - val_acc: 0.4985 - val_binary_accuracy: 0.6938
Epoch 2/15
12000/12000 [==============================] - 4s 321us/step - loss: 0.1960 - acc: 0.5027 - binary_accuracy: 0.6998 - val_loss: 0.1883 - val_acc: 0.5295 - val_binary_accuracy: 0.7074
Epoch 3/15
12000/12000 [==============================] - 4s 325us/step - loss: 0.1844 - acc: 0.5492 - binary_accuracy: 0.7199 - val_loss: 0.1878 - val_acc: 0.5355 - val_binary_accuracy: 0.7093
Epoch 4/15
12000/12000 [==============================] - 4s 326us/step - loss: 0.1751 - acc: 0.5858 - binary_accuracy: 0.7369 - val_loss: 0.1866 - val_acc: 0.5402 - val_binary_accuracy: 0.7158
Epoch 5/15
12000/12000 [==============================] - 4s 324us/step - loss: 0.1633 - acc: 0.6270 - binary_accuracy: 0.7608 - val_loss: 0.1897 - val_acc: 0.5368 - val_bina

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 11s 899us/step - loss: 0.2186 - acc: 0.4273 - binary_accuracy: 0.6605 - val_loss: 0.1953 - val_acc: 0.4918 - val_binary_accuracy: 0.6972
Epoch 2/15
12000/12000 [==============================] - 4s 325us/step - loss: 0.1958 - acc: 0.5037 - binary_accuracy: 0.6987 - val_loss: 0.1894 - val_acc: 0.5192 - val_binary_accuracy: 0.7042
Epoch 3/15
12000/12000 [==============================] - 4s 316us/step - loss: 0.1869 - acc: 0.5358 - binary_accuracy: 0.7157 - val_loss: 0.1876 - val_acc: 0.5362 - val_binary_accuracy: 0.7070
Epoch 4/15
12000/12000 [==============================] - 4s 311us/step - loss: 0.1763 - acc: 0.5803 - binary_accuracy: 0.7344 - val_loss: 0.1870 - val_acc: 0.5422 - val_binary_accuracy: 0.7125
Epoch 5/15
12000/12000 [==============================] - 4s 307us/step - loss: 0.1647 - acc: 0.6208 - binary_accuracy: 0.7569 - val_loss: 0.1905 - val_acc: 0.5358 - val_bina

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 11s 919us/step - loss: 0.2201 - acc: 0.4283 - binary_accuracy: 0.6613 - val_loss: 0.1939 - val_acc: 0.5025 - val_binary_accuracy: 0.7010
Epoch 2/15
12000/12000 [==============================] - 4s 303us/step - loss: 0.1985 - acc: 0.4950 - binary_accuracy: 0.6971 - val_loss: 0.1940 - val_acc: 0.5052 - val_binary_accuracy: 0.6991
Epoch 3/15
12000/12000 [==============================] - 4s 310us/step - loss: 0.1887 - acc: 0.5282 - binary_accuracy: 0.7121 - val_loss: 0.1863 - val_acc: 0.5492 - val_binary_accuracy: 0.7139
Epoch 4/15
12000/12000 [==============================] - 4s 305us/step - loss: 0.1779 - acc: 0.5732 - binary_accuracy: 0.7313 - val_loss: 0.1870 - val_acc: 0.5465 - val_binary_accuracy: 0.7132
Epoch 5/15
12000/12000 [==============================] - 4s 305us/step - loss: 0.1636 - acc: 0.6231 - binary_accuracy: 0.7569 - val_loss: 0.1926 - val_acc: 0.5335 - val_bina

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 20s 2ms/step - loss: 0.2172 - acc: 0.4355 - binary_accuracy: 0.6619 - val_loss: 0.1920 - val_acc: 0.5095 - val_binary_accuracy: 0.7047
Epoch 2/15
12000/12000 [==============================] - 6s 483us/step - loss: 0.1935 - acc: 0.5172 - binary_accuracy: 0.7035 - val_loss: 0.1878 - val_acc: 0.5392 - val_binary_accuracy: 0.7075
Epoch 3/15
12000/12000 [==============================] - 6s 524us/step - loss: 0.1827 - acc: 0.5557 - binary_accuracy: 0.7219 - val_loss: 0.1875 - val_acc: 0.5372 - val_binary_accuracy: 0.7071
Epoch 4/15
12000/12000 [==============================] - 6s 501us/step - loss: 0.1717 - acc: 0.6003 - binary_accuracy: 0.7426 - val_loss: 0.1887 - val_acc: 0.5422 - val_binary_accuracy: 0.7065
Epoch 5/15
12000/12000 [==============================] - 6s 488us/step - loss: 0.1556 - acc: 0.6473 - binary_accuracy: 0.7744 - val_loss: 0.1979 - val_acc: 0.5242 - val_binary